In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
def should_plot_as_log(measure_name):
    patterns = [
        "loss",
#         "weights_product",
    ]
    
    for p in patterns:
        if p in measure_name:
            return True
    
    return False

def plot_measure(ms, measure_name, groupby, xlabel=None):
    ms = ms.copy()
    if should_plot_as_log(measure_name):
        log_measure_name = "log10_{}".format(measure_name)
        ms[log_measure_name] = np.log10(ms[measure_name])
        measure_name = log_measure_name

    sns.boxplot(data=ms, x=measure_name, y=groupby, orient="h")
    swarm_plot = sns.swarmplot(data=ms, x=measure_name, y=groupby,
                      size=2, color=".3", linewidth=0, orient="h",
                 )
    
    if xlabel:
        swarm_plot.set(xlabel=xlabel)
    
    return swarm_plot.get_figure()

def plot_all_measures(ms, groupby="kind"):
    for g_id, g in ms.groupby(groupby):
        print(g_id, g.count().iloc[0])

    for measure in [
                "loss_train", "loss_test",
                "gradient_norm_test",
                "gradient_norm_squared_test",
                "weights_product",
                "path_length_f_test",
                "path_length_d_test",
                "actual_epochs",
            ]:
        plot_measure(ms, measure, groupby)
        plt.show()

In [ ]:
ms = smooth.analysis.load_measures(
    "./0410_gradient_reg/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.gradient_norm_reg_coef", "gn"),
    ],
)

plot_all_measures(ms)

In [ ]:
ms1 = ms[ms["model.learning_rate"] == 0.01].copy()
ms1["Regularization coef"] = ms1["model.gradient_norm_reg_coef"]

ms1 = smooth.analysis.get_ratios(ms1, ms1["model.gradient_norm_reg_coef"] == 0, "gradient_norm_test")

fig1 = plot_measure(ms1, "gradient_norm_test", groupby="Regularization coef", xlabel="Gradient norm")
fig1.show()
fig1.savefig("../figures/teatalk_gradient_norm.png", bbox_inches="tight")
# plot_all_measures(ms1, "Regularization coef")

In [ ]:
fig2 = plot_measure(ms1, "gradient_norm_test_normed",
                    groupby="Regularization coef", xlabel="Gradient norm - normalized")
fig2.show()
fig2.savefig("../figures/teatalk_gradient_norm_normalized.png", bbox_inches="tight")

In [ ]:
ms = smooth.analysis.load_measures(
    "./0410_pca/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.pca_dims", "pca"),
    ],
)
ms = ms[ms["model.pca_dims"] != 49]
plot_all_measures(ms)

In [ ]:
ms = smooth.analysis.load_measures(
    "./0412_weights_product_reg/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.weights_product_reg_coef", "wp"),
    ],
)
ms = ms[ms["model.weights_product_reg_coef"] < 1e-4]

ms1 = smooth.analysis.load_measures(
    "./0413_weights_product_reg/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.weights_product_reg_coef", "wp"),
    ],
)
ms = pd.concat([ms, ms1], sort=False)
ms = ms.sort_values(["model.learning_rate", "model.weights_product_reg_coef"])

# ms = ms.reset_index(drop=True)
# ms.to_feather("0413_weights_product_reg/measures_both.feather")
# plot_all_measures(ms)

In [ ]:
ms1 = ms[ms["model.learning_rate"] == 0.01].copy()

ms1 = ms1[ms1["dataset.name"].isin(
    set(ms1.loc[ms1["model.weights_product_reg_coef"] == 3e-5, "dataset.name"])
)]

ms1 = smooth.analysis.get_ratios(ms1, ms1["model.weights_product_reg_coef"] == 0, "weights_product")

ms1["Regularization coef"] = ms1["model.weights_product_reg_coef"].map(str)

fig = plot_measure(
    ms1,
    "weights_product",
    groupby="Regularization coef",
    xlabel="Weights product",
)
fig.show()
# ax.set(xlabel='common xlabel', ylabel='common ylabel')
# fig.savefig("../figures/teatalk_weights_product.png", bbox_inches="tight")
# plot_all_measures(ms1, "Regularization coef")

In [ ]:
fig = plot_measure(
    ms1,
    "weights_product_normed",
    groupby="Regularization coef",
    xlabel="Weights product - normed",
)
fig.show()
fig.savefig("../figures/teatalk_weights_product_normalized.png", bbox_inches="tight")

In [ ]:
ms = smooth.analysis.load_measures(
    "./0514_gradient_squared_reg/measures.feather",
    kind_cols=[
#         ("model.learning_rate", "lr"),
        ("model.gradient_norm_squared_reg_coef", "gn"),
    ],
)

plot_all_measures(ms)

In [ ]:
ms["foo"] = ms["gradient_norm_squared_test"] ** 0.5 / ms["gradient_norm_test"]

In [ ]:
plot_measure(ms, "foo", "kind")

In [ ]:
ms = load_measures(
    "./0413-211129/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.weights_product_reg_coef", "wp"),
    ],
)
plot_all_measures(ms)

In [ ]:
dataset = smooth.datasets.from_params("mnist12")

In [ ]:
model = smooth.model.train_shallow(
    dataset,
    learning_rate=0.01,
    init_scale=0.01,
    hidden_size=256,
    epochs=100,
    batch_size=64,
    verbose=1,
    weights_product_reg_coef=1,
)

In [ ]:
model = smooth.model.train_shallow(
    dataset,
    learning_rate=0.01,
    init_scale=0.01,
    hidden_size=256,
    epochs=100,
    batch_size=64,
    verbose=1,
    gradient_norm_reg_coef=1e-2,
    error_threshold=0.06,
)

In [ ]:
layer = tf.keras.layers.Dense(1, weights=[np.array([[2], [1]]), np.array([0])])

In [ ]:
layer(np.array([[3, 4]]))

In [ ]:
smooth.measures.gradient_norm(layer, np.array([[3., 4.]], dtype=np.float32))